# SUP ML 3 - PREDICT

# Librerias

In [1]:
try:
    import catboost
except ImportError:
    !pip install catboost
    import catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 8.7 MB/s eta 0:00:00


In [4]:
pip install unidecode

In [5]:
# Datos
import numpy as np
import pandas as pd
import pickle

# Preprocessing
from unidecode import unidecode
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

# Visualizacion
import matplotlib.pyplot as plt
import seaborn as sns

# inhabilita warnings
import warnings
warnings.filterwarnings("ignore")

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Carga modelo

* Cargar el modelo
* Obtener la lista de model features

In [7]:
model = pickle.load(open('/content/drive/MyDrive/DSC N°1123 - Entregable 2 - Julià_Tizzard_Quenti/data/best_model.txt', 'rb'))
print('MODEL: CatBoostClassifier','\n','Model features list:','\n',model.feature_names_)

MODEL: CatBoostClassifier 
 Model features list: 
 ['mou', 'totmrc', 'change_mou', 'drop_vce', 'drop_dat', 'blck_vce', 'blck_dat', 'unan_vce', 'unan_dat', 'plcd_dat', 'recv_vce', 'recv_sms', 'custcare', 'ccrndmou', 'threeway', 'mou_cvce', 'mou_cdat', 'mou_rvce', 'owylis_vce', 'mouowylisv', 'iwylis_vce', 'mouiwylisv', 'peak_vce', 'peak_dat', 'mou_peav', 'mou_pead', 'opk_vce', 'opk_dat', 'mou_opkv', 'drop_blk', 'complete', 'callfwdv', 'callwait', 'months', 'uniqsubs', 'actvsubs', 'new_cell', 'asl_flag', 'adjrev', 'adjmou', 'adjqty', 'avgrev', 'avgmou', 'avgqty', 'avg3mou', 'avg3qty', 'avg3rev', 'prizm_social_one', 'dualband', 'hnd_price', 'lor', 'adults', 'income', 'numbcars', 'dwllsize', 'eqpdays', 'num_kids', 'crclscod_A', 'crclscod_AA', 'crclscod_B', 'crclscod_BA', 'crclscod_CA', 'crclscod_DA', 'crclscod_EA', 'crclscod_OTHER', 'crclscod_ZA', 'area_ATLANTIC SOUTH', 'area_CALIFORNIA NORTH', 'area_CENTRAL/SOUTH TEXAS', 'area_CHICAGO', 'area_DALLAS', 'area_DC/MARYLAND/VIRGINIA', 'area_GRE

# Carga PREDICT dataset

In [8]:
sample_predict = pd.read_csv('/content/drive/MyDrive/DSC N°1123 - Entregable 2 - Julià_Tizzard_Quenti/data/telecom_churn_PREDICT.csv') #,index_col=0
print('Rows: ', sample_predict.shape[0], '  Columns: ', sample_predict.shape[1], '\n')
sample_predict.head(2)

Rows:  10000   Columns:  99 



,rev,mou,totmrc,da,ovrmou,ovrrev,vceovr,datovr,roam,change_mou,...,forgntvl,ethnic,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd,eqpdays,Customer_ID
0,30.8350,136.75,29.99,0.2475,1.25,0.500,0.5,0.000,0.0975,48.25,...,1.0,U,U,U,U,U,U,Y,216.0,1090001
1,35.8475,352.75,24.27,0.4950,23.25,9.285,8.7,0.585,1.8000,-352.75,...,0.0,N,U,U,U,U,U,N,101.0,1090002


# ML Preprocessing

In [9]:
# Checkear duplicados en Customer_ID
print('Duplicated rows in Customer_ID:', sample_predict['Customer_ID'].duplicated().sum())
# Columna Customer_ID transformada a índice
df=sample_predict.set_index('Customer_ID')

print('Updated index to Customer_ID')
print('Rows: ', df.shape[0], '  Columns: ', df.shape[1])
df.head(2)

Duplicated rows in Customer_ID: 0
Updated index to Customer_ID
Rows:  10000   Columns:  98


,rev,mou,totmrc,da,ovrmou,ovrrev,vceovr,datovr,roam,change_mou,...,dwllsize,forgntvl,ethnic,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd,eqpdays
Customer_ID,,,,,,,,,,,,,,,,,,,,,
1090001,30.8350,136.75,29.99,0.2475,1.25,0.500,0.5,0.000,0.0975,48.25,...,NaN,1.0,U,U,U,U,U,U,Y,216.0
1090002,35.8475,352.75,24.27,0.4950,23.25,9.285,8.7,0.585,1.8000,-352.75,...,NaN,0.0,N,U,U,U,U,U,N,101.0


In [10]:
# Eliminar columnas con valor único
del(df['infobase'])
print(f'Column Infobase was deleted')
print('N° Rows: ',df.shape[0], '  Columns: ', df.shape[1])

Column Infobase was deleted
N° Rows:  10000   Columns:  97


In [11]:
# Eliminar columnas irrelevantes
del(df['avg6rev'])
del(df['avg6qty'])
del(df['avg6mou'])

In [12]:
# Clasificar variables en numéricas y categóricas
var_cat = df.select_dtypes(include=['object']).columns.tolist()
var_num = df.select_dtypes(include=['int','float']).columns.tolist()
print('Total var_cat:', len(var_cat), '   Total var_num:', len(var_num), '    Total variables:', len(var_cat)+len(var_num))

Total var_cat: 20    Total var_num: 74     Total variables: 94


In [13]:
# Para que todo lo que entren (datos del test, datos nuevos tengan las mismas características), Eliminar los tildes:
df[var_cat] = df[var_cat].apply(lambda x: x.apply(lambda y: unidecode(y) if isinstance(y, str) else y))
print('Accent mark deleted', '\n')

# Transformar todos los str a letras mayúsculas
df[var_cat] = df[var_cat].apply(lambda x: x.str.upper() if x.dtype == 'O' else x)
print('Transformed to uppercase', '\n')

# Eliminar ' area' al final de cada string en la columna area
df['area'] = df['area'].str.replace(' AREA', '')
print('AREA word removed', '\n')

# Cambiar 'UNKW' por 'UNKNOWN', ya que este será un valor a imputar en nulos posteriormente
df['hnd_webcap'] = df['hnd_webcap'].replace('UNKW','UNKNOWN')
print('UNKNOWN instead UNKW in hnd_webcap category', '\n')

# Reemplazar la categoría 'T' por 'U' en la variable dualband
df['dualband'] = df['dualband'].str.replace('T', 'U')
print('U instead T in dualband category', '\n')

Accent mark deleted 

Transformed to uppercase 

AREA word removed 

UNKNOWN instead UNKW in hnd_webcap category 

U instead T in dualband category 



In [14]:
# Se agrupan las columnas kids in num_kids
list_to_kids=['kid0_2','kid3_5','kid6_10','kid11_15','kid16_17']
for i in list_to_kids:
    df[i] = df[i].replace({'U': 0, 'Y': 1})

df['num_kids'] = df['kid0_2'] + df['kid3_5'] + df['kid6_10'] + df['kid11_15'] + df['kid16_17']
drop_cols = list(set(list_to_kids))
df.drop(columns=drop_cols, inplace=True)

print(f'Columns removed: {drop_cols}')
print(f"Column created: ['num_kids']")

Columns removed: ['kid11_15', 'kid16_17', 'kid0_2', 'kid6_10', 'kid3_5']
Column created: ['num_kids']


In [15]:
# Agrupar categorias en las variables categoricas
list_to_group = ['ethnic', 'crclscod']
grouping_dict = {
    'crclscod': ['A', 'AA', 'B', 'BA', 'CA', 'EA', 'DA', 'ZA'],
    'ethnic': ['N', 'H', 'S', 'U', 'G', 'Z', 'O', 'I']
}

for column in list_to_group:
    no_group_values = grouping_dict.get(column, [])
    df[column] = df[column].apply(lambda x: 'OTHER' if x not in no_group_values and not pd.isna(x) else x)

print(f'Categories were grouped for: {list_to_group}\n')

Categories were grouped for: ['ethnic', 'crclscod']



**Imputación de nulos**

In [16]:
# NULOS EN CATEGORICOS
var_cat = df.select_dtypes(include=['object']).columns.tolist()

In [17]:
#Para las variables categóricas value_counts
for col in var_cat:
  print(df[col].value_counts(),'\n',)

U    8102
Y    1187
N     711
Name: new_cell, dtype: int64 

AA       2357
OTHER    1858
BA       1397
A        1394
CA        945
EA        898
DA        468
B         367
ZA        316
Name: crclscod, dtype: int64 

N    6663
Y    3337
Name: asl_flag, dtype: int64 

S    2894
U    2678
C    1774
T    1284
R     519
Name: prizm_social_one, dtype: int64 

LOS ANGELES                 2080
SOUTHWEST                   1181
NEW YORK CITY                805
DC/MARYLAND/VIRGINIA         643
ATLANTIC SOUTH               606
MIDWEST                      455
HOUSTON                      454
CALIFORNIA NORTH             437
CHICAGO                      379
GREAT LAKES                  373
NEW ENGLAND                  362
NORTH FLORIDA                354
CENTRAL/SOUTH TEXAS          337
DALLAS                       320
OHIO                         320
NORTHWEST/ROCKY MOUNTAIN     315
SOUTH FLORIDA                213
TENNESSEE                    183
PHILADELPHIA                 183
Name: area, dty

In [18]:
# cambiamos el formato para que coincida con la nueva imputación de nulos
varcat_null_to_U = ['new_cell','dualband']

for i in varcat_null_to_U:
  df[i] = df[i].replace('U', 'UNKNOWN')

  print(i, "\n", df[i].value_counts(), "\n")

new_cell 
 UNKNOWN    8102
Y          1187
N           711
Name: new_cell, dtype: int64 

dualband 
 Y          6288
N          2778
UNKNOWN     934
Name: dualband, dtype: int64 



In [19]:
df.head()

,rev,mou,totmrc,da,ovrmou,ovrrev,vceovr,datovr,roam,change_mou,...,adults,income,numbcars,HHstatin,dwllsize,forgntvl,ethnic,creditcd,eqpdays,num_kids
Customer_ID,,,,,,,,,,,,,,,,,,,,,
1090001,30.8350,136.75,29.9900,0.2475,1.25,0.500,0.5,0.000,0.0975,48.25,...,1.0,3.0,NaN,C,NaN,1.0,U,Y,216.0,0.0
1090002,35.8475,352.75,24.2700,0.4950,23.25,9.285,8.7,0.585,1.8000,-352.75,...,NaN,NaN,NaN,NaN,NaN,0.0,N,N,101.0,0.0
1090003,30.3275,241.50,39.9900,0.0000,0.00,0.000,0.0,0.000,0.0000,-86.50,...,3.0,6.0,1.0,C,A,0.0,N,Y,262.0,0.0
1090004,154.6925,2297.00,149.9900,4.7025,0.00,0.000,0.0,0.000,0.0000,119.00,...,3.0,6.0,1.0,NaN,A,0.0,S,Y,127.0,0.0
1090005,156.0050,542.25,48.9475,0.7425,70.75,28.295,28.1,0.195,0.0000,195.75,...,1.0,6.0,1.0,C,NaN,0.0,N,Y,37.0,0.0


In [20]:
var_cat = df.select_dtypes(include=['object']).columns.tolist()

In [21]:
# Se imputan los nulos como 'UNKNOWN'
var_cat = df.select_dtypes(include=['object']).columns.tolist()
for i in var_cat:
    df[i].fillna('UNKNOWN', inplace=True)

print(f'{len(var_cat)} Null values transformed to UNKNOWN')

15 Null values transformed to UNKNOWN


Nulos en númericas

In [22]:
df['numbcars'].value_counts()

# Efectivamente no hay valores 0 así que asumimos que los nulos que entran son 0

1.0    2098
2.0    1885
3.0     219
Name: numbcars, dtype: int64

In [23]:
var_num = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [24]:
porcentaje_nulos = (df[(df.columns[df.isnull().any()])].isnull().sum() / len(df)) * 100

# numbcars: 43568 nulos, no hay opcion para cuando se tiene 0 autos, asi que es posible que los nulos hagan referencia a 0 cars
df['numbcars'].fillna(0, inplace = True)
print('We have replaced the nulls with 0 in numbcars')

# Las columnas numéricas se reemplazan con -9999 para poder ser outlier
for i in var_num:
   df[i].fillna(-9999, inplace = True)

print(f'{len(porcentaje_nulos)} columns with null values were updated. Null values were replaced by -9999')

We have replaced the nulls with 0 in numbcars
20 columns with null values were updated. Null values were replaced by -9999


In [25]:
# Check nullos
porcentaje_nulos = (df[(df.columns[df.isnull().any()])].isnull().sum() / len(df)) * 100
print(f'{len(porcentaje_nulos)} Columns with null values')

0 Columns with null values


**Encoding categóricos**

In [26]:
l_oe = ['prizm_social_one','dwllsize']

encoder = OrdinalEncoder()

# Ajustar y transformar la columna 'nivel_educativo'
df[l_oe] = encoder.fit_transform(df[l_oe])

In [27]:
# Si tienen unicamente 2 clases, N: 0, Y: 1 y U or UNKNOWN: -9999
mapeo_asl = {'N': 0, 'Y': 1}
df['asl_flag'] = df['asl_flag'].replace(mapeo_asl)
print(f"Unique values for asl_flag: {df['asl_flag'].unique()}")


mapeo_newcell = {'N': 0, 'Y': 1, 'UNKNOWN': -9999}
df['new_cell'] = df['new_cell'].replace(mapeo_newcell)
print(f"Unique values for new_cell: {df['new_cell'].unique()}")


mapeo_dualband = {'N': 0, 'Y': 1, 'UNKNOWN': -9999}
df['dualband'] = df['dualband'].replace(mapeo_dualband)
print(f"Unique values for dualband: {df['dualband'].unique()}")

mapeo_creditcd = {'N': 0, 'Y': 1, 'UNKNOWN': -9999}
df['creditcd'] = df['creditcd'].replace(mapeo_creditcd)
print(f"Unique values for creditcd: {df['creditcd'].unique()}")

Unique values for asl_flag: [0 1]
Unique values for new_cell: [-9999     1     0]
Unique values for dualband: [    1 -9999     0]
Unique values for creditcd: [    1     0 -9999]


In [28]:
# Actualizar variables en numérica y categóricas
var_cat = df.select_dtypes(include=['object']).columns.tolist()
var_num = df.select_dtypes(include=['int', 'float']).columns.tolist()
print('Total var_cat:', len(var_cat), '   Total var_num:', len(var_num), '    Total variables:', len(var_cat)+len(var_num))

Total var_cat: 9    Total var_num: 81     Total variables: 90


In [29]:
# onehotencoding
df = pd.get_dummies(df, columns=var_cat)
print('Rows: ', df.shape[0], '  Columns: ', df.shape[1])

Rows:  10000   Columns:  143


In [30]:
lista_ohe = df.columns.tolist()

In [31]:
# Eliminar columnas altamente correlacionadas
drop_cols = ['vceovr', 'marital_UNKNOWN', 'plcd_vce', 'ethnic_UNKNOWN', 'rev', 'rv', 'ovrrev', 'totcalls', 'totrev', 'truck', 'inonemin', 'creditcd', 'comp_dat', 'mou_opkd', 'ownrent_UNKNOWN', 'datovr', 'refurb_new_N', 'forgntvl', 'roam', 'comp_vce', 'totmou', 'models', 'change_rev', 'phones', 'attempt', 'ovrmou', 'da', 'cc_mou']

# Eliminar elementos que no están en la lista 'lista_ohe'
drop_cols = [x for x in drop_cols if x in lista_ohe]

df.drop(columns=drop_cols, inplace=True)

print(f'Eliminadas {len(drop_cols)} columnas altamente correlacionadas:{drop_cols}')
print('N° Rows: ',df.shape[0], '  Columns: ', df.shape[1])

Eliminadas 28 columnas altamente correlacionadas:['vceovr', 'marital_UNKNOWN', 'plcd_vce', 'ethnic_UNKNOWN', 'rev', 'rv', 'ovrrev', 'totcalls', 'totrev', 'truck', 'inonemin', 'creditcd', 'comp_dat', 'mou_opkd', 'ownrent_UNKNOWN', 'datovr', 'refurb_new_N', 'forgntvl', 'roam', 'comp_vce', 'totmou', 'models', 'change_rev', 'phones', 'attempt', 'ovrmou', 'da', 'cc_mou']
N° Rows:  10000   Columns:  115


In [32]:
# Eliminar columnas de baja varianza
list_low_var = ['area_UNKNOWN', 'refurb_new_UNKNOWN', 'HHstatin_H']

# Eliminar columnas que no están en la lista 'lista_ohe'
list_low_var = [x for x in list_low_var if x in lista_ohe]

df.drop(columns=list_low_var, inplace=True)
print(f'Eliminadas {len(list_low_var)} columnas con baja varianza :{list_low_var}')
print('Rows: ', df.shape[0], '  Columns: ', df.shape[1])

Eliminadas 1 columnas con baja varianza :['HHstatin_H']
Rows:  10000   Columns:  114


In [33]:
sample_predict = df.copy()

# Check model features

* Comprobar que tenemos en el dataset preprocesado todas las model features, de lo contrario no podremos hacer predict.
* Ordenar las variables en mismo orden que las model features

In [34]:
# Check features matched features model
features_predict = list(sample_predict.columns)
model_features = list(model.feature_names_)

In [35]:
for i in features_predict:
  if i not in model_features:
    print(i)

In [36]:
print('Columnas en dataset predict:',len(features_predict))
print('Variables en modelos:',len(model_features))
print('¿Match?:', model_features == features_predict)

Columnas en dataset predict: 114
Variables en modelos: 114
¿Match?: True


In [37]:
missing_features = [i for i in model_features if i not in features_predict]
print('Variables que faltan en el dataset:\n', missing_features)

Variables que faltan en el dataset:
 []


In [38]:
print(model_features)
print(features_predict)

['mou', 'totmrc', 'change_mou', 'drop_vce', 'drop_dat', 'blck_vce', 'blck_dat', 'unan_vce', 'unan_dat', 'plcd_dat', 'recv_vce', 'recv_sms', 'custcare', 'ccrndmou', 'threeway', 'mou_cvce', 'mou_cdat', 'mou_rvce', 'owylis_vce', 'mouowylisv', 'iwylis_vce', 'mouiwylisv', 'peak_vce', 'peak_dat', 'mou_peav', 'mou_pead', 'opk_vce', 'opk_dat', 'mou_opkv', 'drop_blk', 'complete', 'callfwdv', 'callwait', 'months', 'uniqsubs', 'actvsubs', 'new_cell', 'asl_flag', 'adjrev', 'adjmou', 'adjqty', 'avgrev', 'avgmou', 'avgqty', 'avg3mou', 'avg3qty', 'avg3rev', 'prizm_social_one', 'dualband', 'hnd_price', 'lor', 'adults', 'income', 'numbcars', 'dwllsize', 'eqpdays', 'num_kids', 'crclscod_A', 'crclscod_AA', 'crclscod_B', 'crclscod_BA', 'crclscod_CA', 'crclscod_DA', 'crclscod_EA', 'crclscod_OTHER', 'crclscod_ZA', 'area_ATLANTIC SOUTH', 'area_CALIFORNIA NORTH', 'area_CENTRAL/SOUTH TEXAS', 'area_CHICAGO', 'area_DALLAS', 'area_DC/MARYLAND/VIRGINIA', 'area_GREAT LAKES', 'area_HOUSTON', 'area_LOS ANGELES', 'are

# PREDICT

* predict() y predict_proba()

In [39]:
# Reordena variables
X_pred = sample_predict[model_features]

In [40]:
predictions = model.predict(X_pred)

In [41]:
predict = model.predict(sample_predict)[1]

In [42]:
predict_proba = model.predict_proba(sample_predict)[:,1][1]

In [43]:
print('Clientes que van a abandonar próximo mes (churn=1)',predict)
print('Probabilidad de abandono: {:.2%}'.format(predict_proba))

Clientes que van a abandonar próximo mes (churn=1) 0
Probabilidad de abandono: 37.34%


# Guarda predicciones

* Guardar las predicciones en data path. Cada fila debe estar etiquetada con el ID.

In [44]:
df_predict= pd.DataFrame(predictions, columns=['churn'], index = X_pred.index)

In [45]:
df_predict

,churn
Customer_ID,
1090001,0
1090002,0
1090003,0
1090004,1
1090005,0
...,...
1099996,1
1099997,1
1099998,1


In [46]:
df_predict.to_csv('/content/drive/MyDrive/DSC N°1123 - Entregable 2 - Julià_Tizzard_Quenti/data/final_telecom_churn_PREDICTIONS.csv')